In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.data import imread
import matplotlib.pyplot as plt
import torch.nn as nn 
import torch.nn.functional as F
from torchvision import transforms

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from PIL import Image
import os
import torch
from inceptionresnetv2.pytorch_load import InceptionResnetV2
from pathlib import Path

In [2]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial
# the default is gpu but if gpu is not available,then use cpu
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100
# show which device to use
print('using device:', device)

ship_dir = '/datasets/ee285f-public/airbus_ship_detection/'
train_image_dir = os.path.join(ship_dir, 'train_v2')
test_image_dir = os.path.join(ship_dir, 'test_v2')

using device: cuda


In [3]:
label = pd.read_csv('train_ship_label.csv')

np_la = label.values
np_train = np_la[:,0]
total = len(np_train)
np_label = np_la[:,2]


In [4]:
print(total)
#dtype = 'FloatTensor'
def normalize_MNIST_images(x):
    return 2 * x.astype(np.float64) / 255. - 1

192556


In [5]:
net = InceptionResnetV2().cuda()
criterion = nn.CrossEntropyLoss()


In [6]:
'''
import torchvision.models as models
resnet18 = models.resnet18()
net = resnet18.cuda()
'''

'\nimport torchvision.models as models\nresnet18 = models.resnet18()\nnet = resnet18.cuda()\n'

In [7]:
model_path = Path('InceptionResnetV2.1.tnt')
gamma = 1e-5
optimizer = torch.optim.Adam(net.parameters(), lr=gamma )
if model_path.exists():
    net = torch.load(str(model_path))
    print('load succerssful')

#optimizer = torch.optim.SGD(net.parameters(), lr=gamma , momentum=0.9)
batch_size = 16
epoch = 3
load_flag = 0
NB = (total+batch_size-1)//batch_size
print(NB)
false = 0

load succerssful
12035


In [ ]:
false_0 = 0
false_1 = 0
size = 299,299
for e in range(1):
    loss_epoch = 0
    i=0
    '''
    with open('accurracy_0.log', 'r') as f:
        if load_flag ==0:
            false_0 = int(f.read())
    with open('accurracy_1.log', 'r') as f:
        if load_flag ==0:
            false_1 = int(f.read())
    with open('test_his.log', 'r') as f:
        if load_flag ==0:
            i = int(f.read())
            load_flag = 1
    '''
    
    while i < NB:

        x = torch.zeros([batch_size,3,299,299])
        y = torch.zeros([batch_size])
        #idx = np.arange(i*batch_size:(i+1)*batch_size)
        for j in range(batch_size):

            try:
                img = Image.open(os.path.join(train_image_dir,np_train[(i-1)*batch_size+j]))
                #img = Image.open(os.path.join(train_image_dir,np_train[j]))
                img.thumbnail(size)
                x[j] = transforms.ToTensor()(img)
                y[j] = np_label[(i-1)*batch_size+j]
                #y[j] = np_label[j]
            
            
            except:
                print(np_train[j])
        #np.savez('save\input_'+str(i)+'.npz',inpu = x)

        #print(im)
        inpu = x.to(device=device, dtype=dtype)  

        ltemp = y.to(device=device, dtype=torch.long)
        optimizer.zero_grad()
        output = net.forward(inpu)
        #print(output)
        scores = nn.functional.softmax(output,dim = 1)
        
        for n in range(batch_size):
            if scores[n,0]>scores[n,1] and ltemp[n]==1:
                with open('FP_all.log', 'a') as f:
                    f.write(np_train[(i-1)*batch_size+n]+',')
            elif scores[n,0]<scores[n,1] and ltemp[n]==0:
                with open('FN_all.log', 'a') as f:
                    f.write(np_train[(i-1)*batch_size+n]+',')
                
        #output = torch.exp(output)
        #output = output/torch.sum(output,1) 
        #print(output,scores,ltemp)
        #loss = criterion(scores,ltemp)
        #loss = sum((scores[:,1]-ltemp)**2)
        #print(i,false)
        #print(scores,ltemp)
        #loss.backward()
        # Parameter update 
        #optimizer.step()
        #if i%100 == 0:
            #print(i,loss)
        '''
        if i%100 ==0:
            
            with open('accurracy_0.log', 'w') as f:
                f.write(str(false_0))
            with open('accurracy_1.log', 'w') as f:
                f.write(str(false_1))
                
            with open('test_his.log', 'w') as f:
                f.write(str(i))
            print('save successful')
        '''
            
        i +=1
    print(loss_epoch/NB)

In [ ]:
net.state_dict()